## Regression numerically stable

In [ ]:
# TODO: See if this entire notebook can be combined with gaussian_process-ipynb

import numpy as np
import matplotlib.pyplot as plt
import gaussian_process.GPfunctions as gp
from gaussian_process import GaussianProcess
from gaussian_process.kernels import (
    SquaredExponentialKernel,
    Kernel,
    SymmetricKernel,
    PositiveDefiniteKernel,
    CovarianceFunction,
)

In [ ]:
# Objective function

objectiveFunction = lambda x: -x * np.sin(x)
objectiveFunctionDerivative = lambda x: -x * np.cos(x) - np.sin(x)

X = np.linspace(start=-3.0, stop=3.0, num=1_000)
y = objectiveFunction(X)
g = objectiveFunctionDerivative(X)

rng = np.random.default_rng(1)
training_indices = rng.choice(np.arange(y.size), size=3, replace=False)
X_train, y_train, g_train = (
    X[training_indices],
    y[training_indices],
    g[training_indices],
)

In [ ]:
kernel = SquaredExponentialKernel()

GP_posterior = GaussianProcess(kernel, x_known=X_train, f_known=y_train)

mean, variance = GP_posterior(X)

fig, (ax1) = plt.subplots(1, 1)

gp.plot_objective(ax1, X, y, X_train, y_train)
gp.plot_gp(ax1, X, mean, variance)
gp.plot_label(ax1, "Posterior")

fig.suptitle("Squared exponential kernel without gradient info")
plt.show()

GP_posterior = GaussianProcess(
    kernel=kernel, x_known=X_train, f_known=y_train, g_known=g_train
)

mean, variance = GP_posterior(X)

fig, (ax1) = plt.subplots(1, 1, sharey=True)

gp.plot_objective(ax1, X, y, X_train, y_train)
gp.plot_gp(ax1, X, mean, variance)
gp.plot_label(ax1, "Posterior")

fig.suptitle("Squared exponential kernel with gradient info")
plt.show()

## GP derivative

In [ ]:
kernel = SquaredExponentialKernel()

GP_posterior = GaussianProcess(kernel, X_train, y_train)

mean, variance = GP_posterior.evaluate(X)

mean_derivative, mean_derivative_variance = GP_posterior.derivative(X)

fig, (ax1) = plt.subplots(1, 1, sharey=True)

gp.plot_objective(ax1, X, y, X_train, y_train)
gp.plot_gp(ax1, X, mean, variance)
gp.plot_gp(ax1, X, mean_derivative, mean_derivative_variance)
gp.plot_label(ax1, "Posterior derivative")

fig.suptitle("Squared exponential kernel without gradient info")
plt.show()

GP_posterior = GaussianProcess(kernel, X_train, y_train, g_train)

mean, variance = GP_posterior.evaluate(X)

mean_derivative, mean_derivative_variance = GP_posterior.derivative(X)

fig, (ax1) = plt.subplots(1, 1, sharey=True)

gp.plot_objective(ax1, X, y, X_train, y_train)
gp.plot_gp(ax1, X, mean, variance)
gp.plot_gp(ax1, X, mean_derivative, mean_derivative_variance)
gp.plot_label(ax1, "Posterior derivative")

fig.suptitle("Squared exponential kernel with gradient info")
plt.show()

## Derivative of variance

In [ ]:
kernel = SquaredExponentialKernel()

GP_posterior = GaussianProcess(kernel, X_train, y_train)

variance = GP_posterior.variance(X)
variance_derivative = GP_posterior.derivative_variance(X)

fig, (ax1) = plt.subplots(1, 1, sharey=True)

ax1.plot(X, variance, label="Uncertainty")
ax1.plot(X, variance_derivative, label="Uncertainty derivative")
gp.plot_label(ax1, "Derivative of uncertainty")

fig.suptitle("Squared exponential kernel without gradient info")
plt.show()

GP_posterior = GaussianProcess(kernel, X_train, y_train, g_train)

variance = GP_posterior.variance(X)
variance_derivative = GP_posterior.derivative_variance(X)

fig, (ax1) = plt.subplots(1, 1, sharey=True)

ax1.plot(X, variance, label="Uncertainty")
ax1.plot(X, variance_derivative, label="Uncertainty derivative")
gp.plot_label(ax1, "Derivative of uncertainty")

fig.suptitle("Squared exponential kernel with gradient info")
plt.show()

## Cholesky LDL QR

In [ ]:
class KernelProxy(Kernel):
    def __init__(self, kernel: Kernel) -> None:
        super().__init__()
        self.kernel = kernel

    def evaluate(self, a, b):
        return self.kernel.evaluate(a, b)

    def derivative_lhs(self, a, b):
        return self.kernel.derivative_lhs(a, b)

    def derivative_rhs(self, a, b):
        return self.kernel.derivative_rhs(a, b)

    def derivative_lhsrhs(self, a, b):
        return self.kernel.derivative_lhsrhs(a, b)


class CholeskyKernel(KernelProxy, CovarianceFunction):
    def __init__(self, kernel: Kernel) -> None:
        super().__init__(kernel)


class LDLKernel(KernelProxy, SymmetricKernel):
    def __init__(self, kernel: Kernel) -> None:
        super().__init__(kernel)


class QRKernel(KernelProxy):
    def __init__(self, kernel: Kernel) -> None:
        super().__init__(kernel)


kernel = SquaredExponentialKernel()
GP_cholesky = GaussianProcess(CholeskyKernel(kernel), X_train, y_train, g_train)
GP_ldl = GaussianProcess(LDLKernel(kernel), X_train, y_train, g_train)
GP_qr = GaussianProcess(QRKernel(kernel), X_train, y_train, g_train)

fig, (ax1) = plt.subplots(1, 1, sharey=True)

gp.plot_gp(ax1, X, GP_cholesky.mean(X), GP_cholesky.std_deviation(X), label="Cholesky")
gp.plot_gp(ax1, X, GP_ldl.mean(X), GP_ldl.std_deviation(X), label="LDL")
gp.plot_gp(ax1, X, GP_qr.mean(X), GP_qr.std_deviation(X), label="QR")
gp.plot_label(ax1, "mean")
plt.show()

fig, (ax1) = plt.subplots(1, 1, sharey=True)

gp.plot_gp(
    ax1,
    X,
    GP_cholesky.derivative_mean(X),
    GP_cholesky.derivative_std_deviation(X),
    label="Cholesky",
)
gp.plot_gp(
    ax1, X, GP_ldl.derivative_mean(X), GP_ldl.derivative_std_deviation(X), label="LDL"
)
gp.plot_gp(
    ax1, X, GP_qr.derivative_mean(X), GP_qr.derivative_std_deviation(X), label="QR"
)
gp.plot_label(ax1, "derivative")
plt.show()

print(f"LML of cholesky is {GP_cholesky.log_marginal_likelihood()}")
print(f"LML of LDL      is {GP_ldl.log_marginal_likelihood()}")
print(f"LML of QR       is {GP_qr.log_marginal_likelihood()}")